In [ ]:
%load_ext autoreload
%autoreload 2

import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import lib

PROJECT_ROOT = os.path.abspath(os.path.join(os.curdir, os.pardir))
DATA_ROOT = os.path.join(PROJECT_ROOT, 'data')
EXPORT_ROOT = os.path.join(PROJECT_ROOT, 'export')
os.makedirs(EXPORT_ROOT, exist_ok=True)

PROJECT_ROOT

In [ ]:
config = {
    "objects": [
        {"src_bbox": [[338, 95], [907, 95], [910, 660], [330, 660]]},
    ]
}

VIDEO_FNAME = '2_5262567863796895591.mp4'
VIDEO_FPATH = os.path.join(DATA_ROOT, "AnalogDevicesImgs", VIDEO_FNAME)

In [ ]:
reader = lib.VideoReader(VIDEO_FPATH)
frame = reader[0]

roi_poly_pnts = config["objects"][0]["src_bbox"]

screen_extr = lib.IndicatorScreenExtractor(roi_pnts=roi_poly_pnts, ind_size=800)
screen_extr.fit(frame)

In [ ]:
screen_frame = screen_extr.transform(frame)

screen_frame_eq = lib.equalizeHist(screen_frame)

thresh_img = lib.binarizeOtsu(screen_frame)
# thresh_img = lib.adaptiveThreshold(screen_frame, filter_sz=0)
thresh_img = np.invert(thresh_img)

################
_, ax = plt.subplots(ncols=4, figsize=(25, 10))
ax[0].imshow(frame)
ax[1].imshow(screen_frame)
ax[2].imshow(screen_frame_eq)
ax[3].imshow(thresh_img, cmap='gray')

In [ ]:
# Find arrow


THRESHOLD = int(screen_frame.shape[0]*3/4)

# Hough (0-45, 135-180)
lines1 = cv2.HoughLines(thresh_img, rho=1, theta=np.pi / 180, threshold=THRESHOLD, lines=None, srn=0, stn=0, min_theta=0, max_theta=np.pi/4)
lines2 = cv2.HoughLines(thresh_img, rho=1, theta=np.pi / 180, threshold=THRESHOLD, lines=None, srn=0, stn=0, min_theta=np.pi*3/4, max_theta=np.pi)

lines = []

if lines1 is not None:
    lines.append(lines1)

if lines2 is not None:
    lines.append(lines2)

if len(lines) > 0:
    lines = np.concatenate(lines)

def filter_lines_in_middle(frame, lines, y_check=460):
    result = []
    _, imw = frame.shape[:2]

    left_limit = imw*2/5
    right_limit = imw*4/5

    for src_line in lines:
        line = src_line[0]
        rho, theta = line[:2]

        x_check = (rho-y_check*np.sin(theta))/np.cos(theta)

        if x_check < left_limit or x_check > right_limit:
            continue

        result.append(src_line)

    return result

lines = filter_lines_in_middle(screen_frame, lines, y_check=460)

arrow_line = lines[0]

def hough_lines_2_kb(lines):
    result = []
    for line in lines:
        rho, theta = line[:2]
        k = -(np.cos(theta)/np.sin(theta))
        b = rho/np.sin(theta)
        result.append((k, b))
    
    return result

arrow_lines_kb = hough_lines_2_kb(arrow_line)
arrow_line_kb = arrow_lines_kb[0]
print(arrow_line_kb)

screen_canvas = lib.drawLines(screen_frame, lines)
################
_, ax = plt.subplots(ncols=2, figsize=(25, 10))
ax[0].imshow(screen_canvas)
# ax[1].imshow(screen_frame)
# ax[2].imshow(screen_frame_eq)
# ax[3].imshow(thresh_img, cmap='gray')


In [ ]:
contours, hierarchy = cv2.findContours(
    thresh_img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)

# Filter by area
contours = [contour for contour in contours if 40 < cv2.contourArea(contour) < 250]

# Filter by perimeter
contours = [contour for contour in contours if 50 < cv2.arcLength(contour, True) < 200]

# Filter by position (Y)
contours = [contour for contour in contours if cv2.boundingRect(contour)[1] < 180]


# Find lines


def draw_fit_line(canvas, line_params, color):
    _, cols = canvas.shape[:2]

    [vx, vy, x, y] = line_params
    lefty = int((-x * vy / vx) + y)
    righty = int(((cols - x) * vy / vx) + y)
    cv2.line(canvas, (cols - 1, righty), (0, lefty), color, 2)


def draw_fit_line_scipy(canvas, line_params, color):
    _, cols = canvas.shape[:2]

    k, b = line_params

    lefty = int(b)
    righty = int(k*cols+b)

    cv2.line(canvas, (cols - 1, righty), (0, lefty), color, 2)


def draw_fit_line_angle(canvas, line_params, pnt, color):
    _, cols = canvas.shape[:2]

    theta = line_params

    c = np.cos(theta)
    s = np.sin(theta)
    k = -(c/s)
    b = -k*pnt[0]+pnt[1]

    lefty = int(b)
    righty = int(k*cols+b)

    cv2.line(canvas, (cols - 1, righty), (0, lefty), color, 2)

# canvas2 = screen_frame.copy()
# for contour in contours:
#     # contour = contour[:,0]

#     mask = np.zeros(screen_frame.shape[:2], dtype=np.uint8)
#     cv2.drawContours(mask, [contour], 0, 255, 3, cv2.LINE_AA, None, 1)

#     line = cv2.fitLine(contour, cv2.DIST_L2, 0, 0.1, 0.1)
#     draw_fit_line(canvas2, line, color=(0, 205, 0))


from scipy.optimize import curve_fit, minimize as sc_minimize, leastsq
from functools import partial
from benderopt import minimize

def f(x, *p):
    return np.poly1d(p)(x)

def f_angle(x, theta, pnt):
    c = np.cos(theta)
    s = np.sin(theta)
    return ((c/s)*(pnt[0]-x) + pnt[1])

def fit_line(contour, point):
    mask = np.zeros((screen_frame.shape[:2]), dtype=np.uint8)
    cv2.drawContours(mask, [contour], 0, 255, -1)
    res = cv2.findNonZero(mask)

    xs = res[:,0,0]
    ys = res[:,0,1]

    fit_func = partial(f_angle, pnt=point)
    p1, _ = curve_fit(fit_func, xs, ys, p0=(.05))

    return p1, xs, ys

def f_point_from_arrow(y, contours, arrow_line):
    k, b = arrow_line
    x = (y-b)/k

    return f_point(x, y, contours=contours)

def f_point(x, y, contours):
    errors = []
    point = [x, y]

    for contour in contours:
        contour = contour[:,0]

        p1, xs, ys = fit_line(contour, point)

        error = ((f_angle(xs, *p1, pnt=point)-ys)**2).mean()
        errors.append(error)
    
    return np.mean(errors)


optimization_problem = [
    # {
    #     "name": "x",
    #     "category": "uniform",
    #     "search_space": {
    #         "low": 0,
    #         "high": screen_frame.shape[1],
    #     }
    # },
    {
        "name": "y",
        "category": "uniform",
        "search_space": {
            "low": 400,
            "high": screen_frame.shape[0]*3/2,
        }
    },
]

# def fit_point(contours):
minimization_result = minimize(
    # partial(f_point, contours=contours),
    partial(f_point_from_arrow, contours=contours, arrow_line=arrow_line_kb),
    optimization_problem,
    optimizer_type="parzen_estimator",
    number_of_evaluation=300,
    seed=42,
    # debug=True
)

canvas3 = screen_frame.copy()

k, b = arrow_line_kb
opt_y = minimization_result['y']
opt_x = (opt_y-b)/k
opt_point = [opt_x, opt_y]
print(opt_point)

line_angles = []
for contour in contours:
    contour = contour[:,0]

    p1, _, _ = fit_line(contour, opt_point)
    line_angles.append(p1[0])
    draw_fit_line_angle(canvas3, p1, pnt=opt_point, color=(0, 205, 0))

# np.insert(xs, 0, point[0])
# np.insert(ys, 0, point[1])
# sigma = np.ones(len(xs))
# sigma[0] = 0.001

canvas = screen_frame.copy()
cv2.drawContours(canvas, contours, -1, (255, 0, 0), 3, cv2.LINE_AA, None, 1)

################
fig, ax = plt.subplots(ncols=2, figsize=(25, 10))
ax[0].imshow(canvas)
# ax[1].imshow(canvas2)
ax[1].imshow(canvas3)

fig.tight_layout()
fig.savefig(os.path.join(EXPORT_ROOT, "05_marks.png"))

In [ ]:
line_angles = np.array(line_angles)

mask = line_angles > np.pi/2
line_angles[mask] = line_angles[mask] - np.pi

np.rad2deg(sorted(line_angles))



In [ ]:
diffs = np.diff(sorted(line_angles))
avg_key_step = np.median(diffs)

diffs, avg_key_step

In [ ]:
# 1 - step required to 
value_step = 1
# 2 - most left value
most_left_value = 1

X = [sorted(line_angles)[0]]
y = [most_left_value]

prev_value = None
cur_value = most_left_value
for rad_angle in sorted(line_angles):
    if prev_value is None:
        prev_value = rad_angle
        continue

    diff = np.abs(rad_angle-prev_value)

    n_steps = round(diff/avg_key_step)

    cur_value += n_steps*value_step

    X.append(rad_angle)
    y.append(cur_value)

    prev_value = rad_angle


In [ ]:
z = np.polyfit(x=X, y=y, deg=1)
poly = np.poly1d(z)

print(poly)
print(poly(0))